<a href="https://colab.research.google.com/github/patbaa/demo_notebooks/blob/master/first_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Notebook to train your first CNN on the cifar 10 dataset


In [0]:
%tensorflow_version 2.x

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid


from tensorflow import keras
from tensorflow.keras.layers import *

%matplotlib inline

## Load data and check dimensions plus visualize a few images

## The CIFAR10 dataset consists of 10 categories:
 - airplane
 - automobile
 - bird
 - cat
 - deer
 - dog
 - frog
 - horse
 - ship
 - truck

Each ismage is 32x32 pixels and RGB. Because of the size of the images and the categories, the task is not as easy as it was for the MNIST!

In [0]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print(f'x_train shape: {x_train.shape}')
print(f'{x_train.shape[0]} train samples')
print(f'{x_test.shape[0]} test samples')

In [0]:
cifar10classes = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 
                  5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}
fig = plt.figure(1, (15., 4.))
grid = ImageGrid(fig, 111, nrows_ncols=(3, 12), axes_pad=0.4)

for i in range(36):
    grid[i].imshow(x_train[i])
    grid[i].set_title(cifar10classes[y_train[i][0]])
    grid[i].axis('off')

Let's convert the labels to one-hot encoded labels and scale the pixels to 0-1!

In [0]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test  = keras.utils.to_categorical(y_test,  10)

In [0]:
x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')
x_train /= 255
x_test  /= 255

Our model will be a VGG-like convolutional neural network. Two convolutional layers and a maxpooling consists of a block. Three of these blocks are followed two fully connected layers.

In [0]:
cnn_model = keras.models.Sequential()
cnn_model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:]))
cnn_model.add(Activation('relu'))
cnn_model.add(Conv2D(32, (3, 3)))
cnn_model.add(Activation('relu'))
cnn_model.add(MaxPooling2D())

cnn_model.add(Conv2D(64, (3, 3)))
cnn_model.add(Activation('relu'))
cnn_model.add(Conv2D(64, (3, 3)))
cnn_model.add(Activation('relu'))
cnn_model.add(MaxPooling2D())

cnn_model.add(Conv2D(128, (3, 3)))
cnn_model.add(Activation('relu'))
cnn_model.add(Conv2D(128, (3, 3)))
cnn_model.add(Activation('relu'))
cnn_model.add(Flatten())

cnn_model.add(Dense(128))
cnn_model.add(Activation('relu'))
cnn_model.add(Dense(10))
cnn_model.add(Activation('softmax'))

In [0]:
cnn_model.summary()

In [0]:
cnn_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-4), metrics=['accuracy'])

## Do you have GPU turned on?

In [0]:
cnn_history = cnn_model.fit(x_train, y_train, batch_size=32,
                            epochs=50, validation_data=(x_test, y_test), shuffle=True)

# Fully connected via the functional API

The functinal API allows to wire more flexible neural networks compare to the sequential API. One layer's outout can be redirected to more than one layer's input and the model can handle multiple outputs coming form different layers. Also it is possible to give more inputs to a single model to different layers (eg an image for the convolutional neural network part & corresponding metadata to the fully connected part. The functional API is not necessary here, just an example.

In [0]:
inp = Input(shape=(32*32*3))
x = Dense(2048, activation='relu')(inp)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(10, activation='softmax')(x)
fc_model = keras.models.Model(inputs=inp, outputs=x)

In [0]:
fc_model.summary()

In [0]:
fc_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-4), metrics=['accuracy'])

In [0]:
fc_history = fc_model.fit(x_train.reshape(50000, 32*32*3), y_train, batch_size=32,
                            epochs=50, validation_data=(x_test.reshape(10000, 32*32*3), y_test), shuffle=True)

In [0]:
plt.figure(figsize=(20, 6))
plt.subplot(121)
plt.plot(fc_history.history['accuracy'], label='FC train acc', lw=5, c='r')
plt.plot(fc_history.history['val_accuracy'], label='FC validataion acc', lw=5, c='r', ls='--')
plt.plot(cnn_history.history['accuracy'], label='CNN train acc', lw=5, c='b')
plt.plot(cnn_history.history['val_accuracy'], label='CNN validataion acc', lw=5, c='b', ls='--')
plt.xlabel('#epochs', fontsize=15)
plt.ylabel('accuracy', fontsize=15)
plt.ylim(0.3, 1)
plt.legend(fontsize=15)

plt.subplot(122)
plt.plot(fc_history.history['loss'], label='FC train loss', lw=5, c='r')
plt.plot(fc_history.history['val_loss'], label='FC validataion loss', lw=5, c='r', ls='--')
plt.plot(cnn_history.history['loss'], label='CNN train loss', lw=5, c='b')
plt.plot(cnn_history.history['val_loss'], label='CNN validataion loss', lw=5, c='b', ls='--')
plt.xlabel('#epochs', fontsize=15)
plt.ylabel('loss', fontsize=15)
plt.legend(fontsize=15)
plt.show()

# What can we conclude from the learning curves?